In [1]:
%matplotlib inline

import os

import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Input, Activation, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Convolution3D, MaxPooling3D, GlobalAveragePooling3D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

from tensorflow.examples.tutorials.mnist import input_data

c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = n

In [2]:
def Shortcut3D(input_data, fx, data_format = "channels_first"):

  # チャンネル数の取得
  if data_format == "channels_first":
    channel_num = int(fx.shape[1])
  else:
    channel_num = int(fx.shape[4])
      
  # inputs と residual とでチャネル数が違うかもしれない。
  # そのままだと足せないので、1x1 conv を使って residual 側のフィルタ数に合わせている
  buf = Convolution3D(channel_num, (1,1,1), strides=(1,1,1), padding='valid', data_format=data_format)(input_data)

  print("fx.shape = {0}, buf.shape = {1}".format(fx.shape, buf.shape))

  # 2つを足す
  return Add()([buf, fx])

In [3]:
def Resblock3D(n_filters, strides=(1,1,1), data_format = "channels_first", axis=1):
  def f(input_data):   
    print("input_data.shape = {0}".format(input_data.shape))
    fx = Convolution3D(n_filters, (3,3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(input_data)
    fx = BatchNormalization(axis=axis)(fx)
    fx = Activation('relu')(fx)
    fx = Convolution3D(n_filters, (3,3,3), strides=strides, kernel_initializer='he_normal', padding='same', data_format=data_format)(fx)
    fx = BatchNormalization(axis=axis)(fx)

    return Shortcut3D(input_data, fx, data_format)

  return f

In [4]:
def Resnet3D(is_channels_first = True):
  if is_channels_first:
    data_format = "channels_first"
    axis=1
    input_data = Input(shape=(1, 3, 28, 28))
  else:
    data_format = "channels_last"
    axis=-1
    input_data = Input(shape=(3, 28, 28, 1))
  
  x = Convolution3D(32, (7,7,7), strides=(1,1,1), kernel_initializer='he_normal', padding='same', data_format=data_format)(input_data)
  x = BatchNormalization(axis=axis)(x)
  x = Activation('relu')(x)
  x = MaxPooling3D((3, 3,3), strides=(2,2,2), padding='same', data_format=data_format)(x)


  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=64, data_format = data_format, axis=axis)(x)
  print(x.shape)
  x = MaxPooling3D(strides=(2,2,2), data_format=data_format)(x)  
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)
  x = Resblock3D(n_filters=128, data_format = data_format, axis=axis)(x)


  x =  GlobalAveragePooling3D(data_format=data_format)(x)
  x = Dense(10, kernel_initializer='he_normal', activation='softmax')(x)

  model = Model(inputs=input_data, outputs=x)
  return model

In [5]:
model = Resnet3D(is_channels_first=False)
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

W1006 20:10:58.110271 11496 deprecation.py:506] From c:\users\tfukushima1\appdata\local\continuum\anaconda3\envs\techacademy\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


input_data.shape = (?, 2, 14, 14, 32)
fx.shape = (?, 2, 14, 14, 64), buf.shape = (?, 2, 14, 14, 64)
input_data.shape = (?, 2, 14, 14, 64)
fx.shape = (?, 2, 14, 14, 64), buf.shape = (?, 2, 14, 14, 64)
input_data.shape = (?, 2, 14, 14, 64)
fx.shape = (?, 2, 14, 14, 64), buf.shape = (?, 2, 14, 14, 64)
(?, 2, 14, 14, 64)
input_data.shape = (?, 1, 7, 7, 64)
fx.shape = (?, 1, 7, 7, 128), buf.shape = (?, 1, 7, 7, 128)
input_data.shape = (?, 1, 7, 7, 128)
fx.shape = (?, 1, 7, 7, 128), buf.shape = (?, 1, 7, 7, 128)
input_data.shape = (?, 1, 7, 7, 128)
fx.shape = (?, 1, 7, 7, 128), buf.shape = (?, 1, 7, 7, 128)
